In [120]:
# Importing working libraries
import os
import cv2
import copy
import numpy as np
from google.colab import drive # Comment when working in Visual Studio Code
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow # colab do not support cv2.imshow()

# Import from my google drive
drive.mount('/content/drive') # Comment when working in Visual Studio Code

# Loaded the main project folder
project_path = "/content/drive/MyDrive/Colab Notebooks/Personal Project/Road Sign Detection"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
# Chemin utile pour charger le modèle YOLO
darknet_path = os.path.join(project_path,"pre-trained YOLO","darknet")
labels_file = os.path.join(darknet_path,"data","coco.names")
config_file = os.path.join(darknet_path,"cfg","yolov3.cfg")
weights_file = os.path.join(darknet_path,"weights","yolov3.weights")

In [122]:
# Liste des classes (noms des objets détectés)
labels = []
with open(labels_file, "r") as f:
    labels = f.read().strip().split('\n')

# Définir un dictionnaire pour mapper chaque classe à une couleur
class_colors = {}
for i, class_name in enumerate(labels):
    hue = int(360 * i / len(labels))  # Répartition des couleurs sur le cercle chromatique 360
    color = np.array([hue,255,255], dtype=np.uint16)
    color = cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_HSV2BGR)[0][0]
    class_colors[class_name] = tuple(int(c) for c in color)

In [123]:
# Charger les images
images_name = ["desk.jpg","dog.jpg","eagle.jpg","giraffe.jpg","horses.jpg","kite.jpg","person.jpg","scream.jpg"]
images_file = [os.path.join(darknet_path,"data",image_name) for image_name in images_name]
images = [cv2.imread(image_file) for image_file in images_file]

In [124]:
# Charger le modèle YOLO pré-entraîné
net = cv2.dnn.readNet(weights_file,config_file)
# Définir les noms des couches de sortie
output_layer_names = net.getUnconnectedOutLayersNames()

In [125]:
def detection_of_all_boxes(image):
    # Redimensionner l'image et normaliser les valeurs des pixels
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    # Charger l'image dans le réseau
    net.setInput(blob)
    outputs = net.forward(output_layer_names)
    # Initialiser des listes pour les boîtes de détection, les confiances et les classes
    boxes = []
    confidences = []
    class_ids = []
    # Parcourir toutes les sorties du modèle
    for output in outputs:
        for detection in output:
            scores = detection[5:]  # Les scores de confiance pour chaque classe
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Seuil de confiance
                # Coordonnées de la boîte de détection
                center_x = int(detection[0] * image.shape[1])
                center_y = int(detection[1] * image.shape[0])
                width = int(detection[2] * image.shape[1])
                height = int(detection[3] * image.shape[0])
                # Coordonnées de l'angle supérieur gauche de la boîte
                x = int(center_x - width / 2)
                y = int(center_y - height / 2)
                # Ajout des éléments au listes
                boxes.append([x, y, width, height])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    # Return
    return(boxes,confidences,class_ids)

def non_maximum_suppression(boxes,confidences):
    # Appliquer la suppression non maximale
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return(indices)

def draw_bounding_boxes(image,boxes,confidences,class_ids,indices):
    # Copier l'image afin d'affecter des modifications à sa copie
    image_copied = copy.deepcopy(image)
    # Dessiner les boîtes de détection sur l'image d'origine
    for i in range(len(boxes)):
        if i in indices:
            x, y, w, h = boxes[i]
            label = labels[class_ids[i]]
            confidence = confidences[i]
            if label in class_colors:
                color = class_colors[label]
            else:
                color = (0, 0, 0)  # Couleur par défaut (noir)
            cv2.rectangle(image_copied, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image_copied, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    # Return
    return(image_copied)

In [126]:
# Détection grâce au modèle
images_with_detection = []
for image in images:
  boxes,confidences,class_ids = detection_of_all_boxes(image)
  indices = non_maximum_suppression(boxes,confidences)
  image_with_detection = draw_bounding_boxes(image,boxes,confidences,class_ids,indices)
  images_with_detection.append(image_with_detection)

In [127]:
# Afficher les détections

# Créez une figure avec deux colonnes de sous-plots
num_images = len(images)
fig, axes = plt.subplots(num_images, 2, figsize=(10, 5 * num_images))

# Parcourez les images et affichez-les dans les deux colonnes avec les titres appropriés
for i in range(num_images):
    ax1 = axes[i, 0]
    ax1.imshow(images[i])
    ax1.set_title("Avant détection")
    ax1.axis('off')

    ax2 = axes[i, 1]
    ax2.imshow(images_with_detection[i])
    ax2.set_title("Après détection")
    ax2.axis('off')

# Ajustez l'espacement entre les sous-plots
plt.tight_layout()

# Affichez la figure
plt.show()

Output hidden; open in https://colab.research.google.com to view.